<a href="https://colab.research.google.com/github/sanjaynagi/AnoFold/blob/main/notebooks/AnoFold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install py3Dmol

!wget https://github.com/gnina/gnina/releases/download/v1.1/gnina
!chmod +x gnina
!./gnina --version

In [ ]:
wkdir =  "./"

import py3Dmol
import pandas as pd
import numpy as np
import plotly.express as px
import requests
import os
import re
import glob
import time

amino_acid_map = {
    'ALA': 'A', 'ARG': 'R', 'ASN': 'N', 'ASP': 'D', 'CYS': 'C',
    'GLN': 'Q', 'GLU': 'E', 'GLY': 'G', 'HIS': 'H', 'ILE': 'I',
    'LEU': 'L', 'LYS': 'K', 'MET': 'M', 'PHE': 'F', 'PRO': 'P',
    'SER': 'S', 'THR': 'T', 'TRP': 'W', 'TYR': 'Y', 'VAL': 'V'
    }

def get_uniprot_data(gene_id):
    uniprot_acc = vectorbase_to_uniprot(gene_id)
    if not uniprot_acc:
        return f"No UniProt accession found for VectorBase ID: {gene_id}"

    # UniProt API endpoint
    base_url = "https://rest.uniprot.org/uniprotkb/search"

    print(f"UniProt accession: {uniprot_acc}")

    # Query parameters
    params = {
        "query": f"accession:{uniprot_acc}",
        "format": "json",
        "fields": "gene_names,protein_name,organism_name,go"
    }

    # Send request with retry mechanism
    max_retries = 3
    for attempt in range(max_retries):
        try:
            response = requests.get(base_url, params=params)
            response.raise_for_status()
            data = response.json()

            if data['results']:
                result = data['results'][0]
                return {
                    "gene_name": result.get('genes', [{}])[0].get('geneName', {}).get('value', 'N/A'),
                    "protein_name": result.get('proteinDescription', {}).get('recommendedName', {}).get('fullName', {}).get('value', 'N/A'),
                    "organism": result.get('organism', {}).get('scientificName', 'N/A'),
                    "function": next((comment['texts'][0]['value'] for comment in result.get('comments', []) if comment['commentType'] == 'FUNCTION'), 'N/A'),
                    "go_terms": [go['id'] for go in result.get('goTerms', [])],
                }
            else:
                return f"No data found for UniProt accession: {uniprot_acc}"

        except requests.exceptions.RequestException as e:
            if attempt == max_retries - 1:
                return f"Error retrieving data: {str(e)}"
            time.sleep(2 ** attempt)  # Exponential backoff

    return "Max retries reached. Unable to retrieve data."

def vectorbase_to_uniprot(gene_id):
    url = "https://rest.uniprot.org/idmapping/run"
    data = {
        "from": "VEuPathDB",
        "to": "UniProtKB",
        "ids": f"VectorBase:{gene_id}"
    }

    response = requests.post(url, data=data)
    response.raise_for_status()
    job_id = response.json()["jobId"]

    status_url = f"https://rest.uniprot.org/idmapping/status/{job_id}"
    while True:
        status_response = requests.get(status_url)
        status_response.raise_for_status()
        status = status_response.json()
        if "jobStatus" in status and status["jobStatus"] in ("RUNNING", "NEW"):
            continue
        elif "results" in status or "failedIds" in status:
            break

    results_url = f"https://rest.uniprot.org/idmapping/stream/{job_id}"
    results_response = requests.get(results_url)
    results_response.raise_for_status()
    results = results_response.json()

    if "results" in results and results["results"]:
        return results["results"][0]["to"]
    else:
        return None

def download_alphafold_pdb(gene_id, output_dir='.'):
    # Convert VectorBase GeneID to UniProt accession
    uniprot_accession = vectorbase_to_uniprot(gene_id)
    if uniprot_accession is None:
        print(f"No UniProt accession found for GeneID: {gene_id}")
        return None

    # Download the PDB file
    response = requests.get(f"https://alphafold.ebi.ac.uk/files/AF-{uniprot_accession}-F1-model_v4.pdb")
    if response.status_code == 200:
        # Create the output directory if it doesn't exist
        os.makedirs(output_dir, exist_ok=True)

        # Save the PDB file
        output_file = os.path.join(output_dir, f"{gene_id}.pdb")
        with open(output_file, 'wb') as f:
            f.write(response.content)

        print(f"Downloaded AlphaFold PDB for {gene_id} (UniProt: {uniprot_accession}) to {output_file}")
        return output_file
    else:
        print(f"Failed to download AlphaFold PDB for {gene_id} (UniProt: {uniprot_accession}). Status code: {response.status_code}")
        return None

def load_pdb(pdb_path):
    # Read and process the PDB file
    df_pdb = pd.read_csv(pdb_path, skiprows=88, header=None)[0].str.split("\s+", expand=True).iloc[:-3,:-1]
    df_pdb.columns = [
        "record", "atom", "atom_name", "amino_acid", "chain_identifier", "codon",
        "x", "y", "z", "occupancy", "b-factor", "element_symbol",
    ]
    df_pdb = df_pdb.query("record == 'ATOM'")
    return df_pdb

def pdb_to_active_site_coords(pdb_path, target_motif, target_codon_in_motif, target_molecule, flanking_size=5):
    df_pdb = load_pdb(pdb_path)

    # Extract unique amino acids
    aa3 = df_pdb[['amino_acid', 'codon']].drop_duplicates()['amino_acid']

    convert = np.vectorize(lambda x: amino_acid_map.get(x, x))
    aa1 = convert(aa3)

    # Find all occurrences of the motif
    aa_sequence = ''.join(aa1)
    motif_matches = list(re.finditer(target_motif, aa_sequence))

    results = []
    for match in motif_matches:
        start, end = match.start(), match.end()

        # Get flanking region
        flanking_start = max(0, start - flanking_size)
        flanking_end = min(len(aa_sequence), end + flanking_size)
        flanking = aa_sequence[flanking_start:start] + '[' + aa_sequence[start:end] + ']' + aa_sequence[end:flanking_end]

        print(f"Motif detected in PDB at codon {start+1}:{end} = {aa_sequence[start:end]}")
        print(f"Flanking sequence: {flanking}")

        # Find coordinates for the target molecule
        target_idx = str(start + 1 + target_codon_in_motif)
        coords = df_pdb.query("codon == @target_idx and atom_name == @target_molecule")

        if not coords.empty:
            coord_array = coords.drop_duplicates('atom_name')[['x', 'y', 'z']].to_numpy()[0].astype(float)
            results.append({
                'start': start + 1,
                'end': end,
                'motif': aa_sequence[start:end],
                'flanking': flanking,
                'coordinates': coord_array,
                'molecule_number': int(coords['atom'].values[0])
            })

    if len(results) > 1:
        print(f"Warning, multiple {len(results)} matching motifs found")

    return results

import requests
import os
import requests
from openbabel import openbabel

def download_ligand(ligand_name, repo_url="https://raw.githubusercontent.com/sanjaynagi/VectorFold/main/ligands", save_path="ligands"):
    """
    Download a ligand file from the GitHub repository.
    
    Parameters:
    - ligand_name (str): The name of the ligand (without the .pdbqt extension).
    - repo_url (str): The base URL of the GitHub repository's raw content.
    - save_path (str): The local directory where the file will be saved. Default is current directory.
    
    Example:
    download_ligand("deltamethrin", "https://raw.githubusercontent.com/sanjaynagi/VectorFold/main/ligands/")
    """
    os.makedirs(f"{save_path}/raw", exist_ok=True)
    file_url = f"{repo_url}/{ligand_name}.pdbqt"
    file_path = f"{save_path}/raw/{ligand_name}.pdbqt"
    
    try:
        response = requests.get(file_url)
        response.raise_for_status()  # Check if the request was successful
        
        with open(file_path, 'wb') as file:
            file.write(response.content)
        print(f"Ligand '{ligand_name}.pdbqt' downloaded successfully.\n")
    except requests.exceptions.RequestException as e:
        print(f"Failed to download ligand: {e}")

def download_and_prepare_ligand(ligand_name, repo_url="https://raw.githubusercontent.com/sanjaynagi/VectorFold/main/ligands", save_path="ligands", pH=7.4):
    """
    Download a ligand file and prepare it by adding hydrogens at a specified pH using OpenBabel.
    
    Parameters:
    - ligand_name (str): The name of the ligand (without the .pdbqt extension).
    - repo_url (str): The base URL of the GitHub repository's raw content.
    - save_path (str): The local directory where the files will be saved.
    - pH (float): The pH at which to add hydrogens. Default is 7.4.
    
    Returns:
    - str: Path to the prepared ligand file.
    """
    import openbabel
    # Download the ligand
    download_ligand(ligand_name, repo_url, save_path)
    
    # Set up OpenBabel conversion
    obConversion = openbabel.OBConversion()
    obConversion.SetInAndOutFormats("pdbqt", "pdbqt")
    
    mol = openbabel.OBMol()
    
    # Read the downloaded PDBQT file
    input_path = f"{save_path}/raw/{ligand_name}.pdbqt"
    obConversion.ReadFile(mol, input_path)
    
    # Add hydrogens at the specified pH
    mol.AddHydrogens(False, True, pH)
    
    # Write the prepared molecule to a new file
    output_path = f"{save_path}/{ligand_name}.pdbqt"
    obConversion.WriteFile(mol, output_path)
    
    print(f"Ligand prepared and saved as '{output_path}'")
    return output_path

def view_pymol(receptor, ligand, docked, receptor_highlight=None, sticks=False):
  v = py3Dmol.view()
  v.addModel(open(receptor).read())
  if sticks:
    v.setStyle({'cartoon':{},'stick':{'radius':.1}})
  else:
    v.setStyle({'cartoon':{}})
  if receptor_highlight:
    for i in range(receptor_highlight-3, receptor_highlight+3):
      v.setStyle({'model': -1, 'serial': i}, {"cartoon": {'color': 'yellow'}, 'stick':{'radius':.3, 'color':'yellow'}})
  v.addModel(open(ligand).read())
  v.setStyle({'model':1},{'stick':{'colorscheme':'dimgrayCarbon','radius':.125}})
  v.addModelsAsFrames(open(docked).read())
  v.setStyle({'model':2},{'stick':{'colorscheme':'greenCarbon'}})
  v.zoomTo({'model':1})
  v.rotate(90)
  v.animate({'interval':5000})
  return v

def run_molecular_docking(gene_id, ligand, custom_active_site_motif=None, custom_target_codon_in_motif=None, custom_target_molecule=None, **kwargs):

  pdb_path  = f"{wkdir}receptors/{gene_id}.pdb"
  ligand_path = f"{wkdir}ligands/{ligand}.pdbqt"

  if not custom_active_site_motif:
      gene_data = get_uniprot_data(gene_id)
      gene_desc = gene_data['protein_name']
      gene_name = gene_data['gene_name']

      print(f"Running molecular docking for {gene_id} | {gene_name} ({gene_desc}), and {ligand}.")
      if 'P450' in gene_desc:
        custom_active_site_motif, custom_target_molecule, custom_target_codon_in_motif = ('blah', 'C', 3)
      elif 'ester hydrolase' in gene_desc:
        custom_active_site_motif, custom_target_molecule, custom_target_codon_in_motif = ('[LIV].G.S.G', 'O', 4)
      else:
        assert "Unknown gene family, custom motif required"

  # download inputs
  if not os.path.exists(pdb_path):
    download_alphafold_pdb(gene_id, output_dir="receptors")
  if not os.path.exists(ligand_path):
    download_ligand(ligand)

  # get active site coords
  res = pdb_to_active_site_coords(
      pdb_path=pdb_path,
      target_motif=custom_active_site_motif, target_molecule=custom_target_molecule, target_codon_in_motif=custom_target_codon_in_motif)
  x,y,z = res[0]['coordinates']

  # run gnina
  if not os.path.exists(f"{wkdir}docking/{gene_id}_{ligand}.log"):
    os.makedirs("docking", exist_ok=True)
    !./gnina -r {pdb_path} -l {ligand_path} --center_x {x} --center_y {y}  --center_z {z} --size_x 20 --size_y 20 --size_z 20 -o {wkdir}docking/{gene_id}_{ligand}.sdf --log {wkdir}docking/{gene_id}_{ligand}.log --seed 0

  return res

def plot_molecular_docking(gene_id, ligand, custom_active_site_motif=None, custom_target_codon_in_motif=None, custom_target_molecule=None, **kwargs):

  pdb_path  = f"{wkdir}receptors/{gene_id}.pdb"
  ligand_path = f"{wkdir}ligands/{ligand}.pdbqt"

  res = run_molecular_docking(gene_id, ligand, custom_active_site_motif, custom_target_codon_in_motif, custom_target_molecule)
  fig = view_pymol(receptor=pdb_path, ligand=ligand_path, docked=f"{wkdir}docking/{gene_id}_{ligand}.sdf", receptor_highlight=res[0]['molecule_number'], **kwargs)

  return fig

## AnoFold - molecular docking with alphafold and gnina

In [ ]:
genes = ['AGAP006227', 'AGAP006228', 'AGAP006723', 'AGAP006724', 'AGAP006725', 'AGAP006726', 'AGAP006727']

In [ ]:
res = run_molecular_docking('AGAP006227', ligand='malathion'),# active_site_motif='[LIV].G.S.G', target_codon_in_motif=4, target_molecule="O")

In [ ]:
for gene in genes:
  res = run_molecular_docking(gene, ligand='malathion'),# active_site_motif='[LIV].G.S.G', target_codon_in_motif=4, target_molecule="O")

### Chimera / HEM / Superimposement

In [ ]:
%pip install chimerax

In [ ]:
def load_reference_p450():


In [ ]:
!pip install ChimeraX

In [ ]:
from chimerax.core.commands import run

# Open your PDB and the reference PDB with the heme
run(session, pdb_path)
run(session, ref_p450_path)

# Assuming your P450 is model #1 and the reference is model #2
# Perform the superposition
run(session, "match #2 to #1")

# Select the heme from the reference model
run(session, "select #2/name HEM")

# Copy the selected heme to your P450 model
run(session, "copy sel models #1")

# Optionally, you can hide or close the reference model
run(session, "hide #2")
# or
# run(session, "close #2")

# Save the result
run(session, "save your_p450_with_heme.pdb models #1")